In [9]:
import os
from pathlib import Path

import duckdb
import pandas as pd
from IPython.core.magic import register_cell_magic

############ Done but exports into the wrong folder ############

cwd = os.getcwd()
print(f"CWD: {cwd}")

pd.set_option("display.max_rows", 20)  # show all rows
pd.set_option("display.max_columns", None)  # show all columns
pd.set_option("display.max_colwidth", None)  # don't truncate text
pd.set_option("display.expand_frame_repr", False)  # don't wrap across lines


# 1) Where to store the DuckDB database file (change if you want it elsewhere)
DB_PATH = Path("./plt.duckdb")
DB_PATH.parent.mkdir(parents=True, exist_ok=True)
# print(DB_PATH)
con = duckdb.connect(str(DB_PATH))


@register_cell_magic
def ducksql(line, cell):
    res = con.sql(cell)
    try:
        display(res.df())
    except Exception:
        print("OK")

CWD: /Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/python_scripts/other_file


In [10]:
import csv
from pathlib import Path

import duckdb

# 1) Where to write files
SEED_DIR = Path("./dbt/seeds/split_files")
SEED_DIR.mkdir(parents=True, exist_ok=True)


import os

print(os.getcwd())


# 2) Get years
years = [
    r[0]
    for r in con.sql("""
    SELECT DISTINCT CAST(year AS INT)
    FROM staging.transfers_all
    WHERE year IS NOT NULL
    ORDER BY 1
""").fetchall()
]

# 3) Ensure target schema
con.sql("CREATE SCHEMA IF NOT EXISTS marts")

# 4) Build per-year tables + export
for y in years:
    con.sql(f"""
        CREATE OR REPLACE TABLE marts.transfers_{y} AS
        SELECT *
        FROM staging.transfers_all
        WHERE CAST(year AS INT) = {y}
    """)
    out = SEED_DIR / f"premier_league_{y}.csv"

    # EASIEST: let DuckDB write the CSV (fast, correct headers)
    con.execute(f"""
        COPY (SELECT * FROM marts.transfers_{y})
        TO '{out.as_posix()}'
        (HEADER, DELIMITER ',');
    """)

    print(f"wrote {out}")

/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/python_scripts/other_file


CatalogException: Catalog Error: Table with name transfers_all does not exist!
Did you mean "temp.information_schema.character_sets"?

In [11]:
try:
    years = con.sql(
        "SELECT DISTINCT CAST(year AS INT) y FROM staging.transfers_all WHERE year IS NOT NULL ORDER BY y"
    ).fetchall()
    source_view = "staging.transfers_all"
    year_col = "year"
except Exception:
    con.sql("""
        CREATE OR REPLACE VIEW staging.transfers_all_w_year AS
        SELECT *, CAST(strftime(try_cast(transfer_date AS DATE), '%Y') AS INT) AS yr
        FROM staging.transfers_all
    """)
    years = con.sql(
        "SELECT DISTINCT yr AS y FROM staging.transfers_all_w_year WHERE yr IS NOT NULL ORDER BY y"
    ).fetchall()
    source_view = "staging.transfers_all_w_year"
    year_col = "yr"

con.sql("CREATE SCHEMA IF NOT EXISTS marts;")
for (y,) in years:
    con.sql(
        f"CREATE OR REPLACE TABLE marts.transfers_{y} AS SELECT * FROM {source_view} WHERE {year_col} = {y}"
    )
    print(f"Created/updated marts.transfers_{y}")

    csv_file_path = f"./dbt/seeds/split_files/premier_league_{y}.csv"

    try:
        con.execute(f"SELECT * FROM marts.transfers_{y}")
        rows = cur.fetchall()

        #     cur.close()
        # Continue only if there are rows returned.
        if rows:
            # New empty list called 'result'. This will be written to a file.
            result = list()

            # The row name is the first entry for each entity in the description tuple.
            column_names = list()
            # for i in cur.description:
            #     column_names.append(i[0])

            result.append(column_names)
            for row in rows:
                result.append(row)

            # Write result to file.
            with open(csv_file_path, "w", newline="") as csvfile:
                csvwriter = csv.writer(
                    csvfile, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL
                )
                for row in result:
                    csvwriter.writerow(row)
            print(f"File {csv_file_path} created")
    except:
        sys.exit(f"No rows found for query: {y}")
    finally:
        pass

CatalogException: Catalog Error: Schema with name staging does not exist!